# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

import pandas as pd
from sqlalchemy import create_engine
import numpy as np
from sklearn.metrics import fbeta_score, make_scorer, SCORERS
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report, accuracy_score, recall_score, precision_score
import warnings
warnings.filterwarnings('ignore')
import re
import pickle
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [8]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * FROM DisasterResponse",engine)
X = df["message"].values
Y = (df[['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 
       'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 
       'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 
       'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']].values)

### 2. Write a tokenization function to process your text data

In [9]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
def tokenize(text):
    
    #Converting everything to lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    #Tokenize words
    words = word_tokenize(text)
    
    #Removing stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    
    #Reduce words to their stems
    stemmed = [PorterStemmer().stem(w) for w in words]
    
    #Lemmatize verbs by specifying pos
    lemmed = [WordNetLemmatizer().lemmatize(w, pos='v') for w in stemmed]
    
    return lemmed


In [11]:
def display_results(Y_test, Y_pred):
    labels = np.unique(Y_pred)
    confusion_mat = confusion_matrix(Y_test, Y_pred, labels=labels)
    accuracy = (Y_pred == Y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [12]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(estimator = RandomForestClassifier(n_jobs =-1)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1)

np.random.seed(13)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
# predict on test data
Y_pred = pipeline.predict(X_test)

In [22]:
# display results
display_results(Y_test[0], Y_pred[0])

Labels: [0 1]
Confusion Matrix:
 [[32  0]
 [ 1  3]]
Accuracy: 0.972222222222


### 6. Improve your model
Use grid search to find better parameters. 

In [26]:
parameters = {}
parameters["clf__estimator__oob_score"] = [True]
parameters["clf__estimator__n_estimators"] = [10,20,50,100]

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1)
cv.fit(X_train, Y_train) 



GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'clf__estimator__oob_score': [True], 'clf__estimator__n_estimators': [10, 20, 50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [28]:
# Perform grid search on the classifier using 'scorer' as the scoring method using GridSearchCV()
grid_obj = GridSearchCV(pipeline, parameters)

# Fit the grid search object to the training data and find the optimal parameters using fit()
grid_fit = grid_obj.fit(X_train,Y_train)

# Get the estimator
best_clf = grid_fit.best_estimator_ 

In [29]:
# Make predictions using the unoptimized and model
predictions = Y_pred
best_predictions = best_clf.predict(X_test)

In [30]:
display_results(Y_test[0], best_predictions[0])

Labels: [0 1]
Confusion Matrix:
 [[32  0]
 [ 0  4]]
Accuracy: 1.0


In [33]:
# Report the before-and-afterscores
print("Unoptimized model\n------")
print("Accuracy score on testing data: {:.4f}".format(accuracy_score(Y_test[0], predictions[0])))
print("F-score on testing data: {:.4f}".format(fbeta_score(Y_test[0], predictions[0], beta = 0.5, average = "micro")))
print("\nOptimized Model\n------")
print("Final accuracy score on the testing data: {:.4f}".format(accuracy_score(Y_test[0], best_predictions[0])))
print("Final F-score on the testing data: {:.4f}".format(fbeta_score(Y_test[0], best_predictions[0], beta = 0.5, average = "micro")))

Unoptimized model
------
Accuracy score on testing data: 0.9722
F-score on testing data: 0.9722

Optimized Model
------
Final accuracy score on the testing data: 1.0000
Final F-score on the testing data: 1.0000


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [34]:
columns = ['related', 'request', 'offer', 'aid_related', 'medical_help', 'medical_products', 
       'search_and_rescue', 'security', 'military', 'child_alone', 'water', 'food', 'shelter', 
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid', 
       'infrastructure_related', 'transport', 'buildings', 'electricity', 'tools', 
       'hospitals', 'shops', 'aid_centers', 'other_infrastructure', 
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather', 'direct_report']

In [35]:
for i,col in enumerate(columns):
    print(col)
    accuracy = accuracy_score(Y_test[i], best_predictions[i])
    precision = precision_score(Y_test[i], best_predictions[i])
    recall = recall_score(Y_test[i], best_predictions[i])
    print("\tAccuracy: %.4f\tPrecision: %.4f\t Recall: %.4f\n" % (accuracy, precision, recall))

related
	Accuracy: 1.0000	Precision: 1.0000	 Recall: 1.0000

request
	Accuracy: 0.9444	Precision: 1.0000	 Recall: 0.7500

offer
	Accuracy: 0.9722	Precision: 1.0000	 Recall: 0.6667

aid_related
	Accuracy: 1.0000	Precision: 1.0000	 Recall: 1.0000

medical_help
	Accuracy: 0.9722	Precision: 0.0000	 Recall: 0.0000

medical_products
	Accuracy: 0.9167	Precision: 0.0000	 Recall: 0.0000

search_and_rescue
	Accuracy: 0.9722	Precision: 0.0000	 Recall: 0.0000

security
	Accuracy: 1.0000	Precision: 1.0000	 Recall: 1.0000

military
	Accuracy: 1.0000	Precision: 1.0000	 Recall: 1.0000

child_alone
	Accuracy: 0.9722	Precision: 0.8571	 Recall: 1.0000

water
	Accuracy: 1.0000	Precision: 1.0000	 Recall: 1.0000

food
	Accuracy: 1.0000	Precision: 0.0000	 Recall: 0.0000

shelter
	Accuracy: 0.7500	Precision: 1.0000	 Recall: 0.1818

clothing
	Accuracy: 0.9444	Precision: 0.6000	 Recall: 1.0000

money
	Accuracy: 0.8889	Precision: 1.0000	 Recall: 0.2000

missing_people
	Accuracy: 0.9722	Precision: 0.0000	 Recall:

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [36]:
grid_fit.best_params_

{'clf__estimator__n_estimators': 100, 'clf__estimator__oob_score': True}

### 9. Export your model as a pickle file

In [ ]:
filename = 'classifier.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.